In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import importlib
import os
import nn.classifier
import nn.unet as unet
import img.dataset as ds
from img import transformer
import color_classes as cc
    
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler, SequentialSampler

from nn.train_callbacks import TensorboardVisualizerCallback, TensorboardLoggerCallback, ModelSaverCallback
from nn.test_callbacks import PredictionsSaverCallback
import helpers

import database.connector as dbcon


ModuleNotFoundError: No module named 'nn'

In [3]:
importlib.reload(nn.unet)
importlib.reload(cc)
importlib.reload(transformer)
importlib.reload(ds)
importlib.reload(nn.classifier)
importlib.reload(nn.train_callbacks)
importlib.reload(nn.test_callbacks)
importlib.reload(dbcon)

<module 'database.connector' from '/home/ditschuk/kitcar/kitcar-gazebo-generation/segmentation/database/connector.py'>

In [8]:
# Hyperparameters
    img_resize = [320, 160]
    batch_size = 2
    epochs = 5
    threshold = 0.5
    validation_size = 0.2
    sample_size = None  # Put None to work on full dataset

In [11]:
net = unet.UNet1024((3, 160,320))
classifier = nn.classifier.CarvanaClassifier(net, epochs)

In [12]:
db_connector = dbcon.SegmentationDatabaseConnector()

In [13]:
db_images = db_connector.load_all(random=True,max_count=20)
file_count = len(db_images)
train_ds = ds.SegmentationImageDataset(db_connector=db_connector,segmentation_images=db_images[int(file_count*validation_size):])
valid_ds = ds.SegmentationImageDataset(db_connector=db_connector,segmentation_images=db_images[:int(file_count*validation_size)])

SELECT * FROM segmentation_dataset
Connected as  kitcar to MySQL database  kitcar
SELECT * FROM segmentation_dataset ORDER BY RAND() LIMIT 20


In [14]:
from multiprocessing import cpu_count
threads = cpu_count()
use_cuda=False

In [15]:
train_loader = DataLoader(train_ds, batch_size,
                              sampler=RandomSampler(train_ds),
                              num_workers=threads,
                              pin_memory=use_cuda)

In [16]:
valid_loader = DataLoader(valid_ds, batch_size,
                              sampler=SequentialSampler(valid_ds),
                              num_workers=threads,
                              pin_memory=use_cuda)

In [17]:
print("Training on {} samples and validating on {} samples "
          .format(len(train_loader.dataset), len(valid_loader.dataset)))


Training on 16 samples and validating on 4 samples 


In [18]:
data_folder = os.environ.get('KITCAR_GAZEBO_GENERATION_PATH') + '/segmentation/'
tb_viz_cb = TensorboardVisualizerCallback(os.path.join(data_folder, 'logs/tb_viz'))
tb_logs_cb = TensorboardLoggerCallback(os.path.join(data_folder, 'logs/tb_logs'))
model_saver_cb = ModelSaverCallback(os.path.join(data_folder, 'output/models/model_' +
                                                     helpers.get_model_timestamp()), verbose=True)
# Testing callbacks
pred_saver_cb = PredictionsSaverCallback(os.path.join(data_folder, 'output/submit.csv.gz'),
                                             [1280,650], threshold)

In [20]:
classifier.train(train_loader, valid_loader, epochs,
                     callbacks=[tb_viz_cb, tb_logs_cb, model_saver_cb])


Epochs 1/5:   0%|          | 0/8 [?]/home/ditschuk/.local/lib/python3.6/site-packages/torch/nn/functional.py:2390: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/home/ditschuk/.local/lib/python3.6/site-packages/torch/nn/functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/ditschuk/.local/lib/python3.6/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/ditschuk/.local/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: 

train_loss = 1.422327, train_acc = 0.410336
val_loss   = 1.440130, val_acc   = 0.421774
Time elapsed = 14s


Epochs 3/5:   0%|          | 0/8 [?]                     

train_loss = 1.313312, train_acc = 0.532001
val_loss   = 1.283221, val_acc   = 0.556992
Time elapsed = 15s


Epochs 4/5:   0%|          | 0/8 [?]                     

train_loss = 1.248813, train_acc = 0.618324
val_loss   = 1.166939, val_acc   = 0.630183
Time elapsed = 15s


Epochs 5/5:   0%|          | 0/8 [?]                     

train_loss = 1.191164, train_acc = 0.651256
val_loss   = 1.099212, val_acc   = 0.781323
Time elapsed = 15s


Epochs 5/5: 100%|██████████| 8/8 [00:00, loss=1.10645, dice_coeff=0.77476]


train_loss = 1.134154, train_acc = 0.763224
val_loss   = 1.050889, val_acc   = 0.798685
Time elapsed = 15s
Model saved in /home/ditschuk/kitcar/kitcar-gazebo-generation/segmentation/output/models/model_2019-08-18_00h52


In [5]:
test_con = dbcon.SegmentationDatabaseConnector('segmentation_test_data')
test_imgs = test_con.load_all()

SELECT * FROM segmentation_test_data
Connected as  kitcar to MySQL database  kitcar
SELECT * FROM segmentation_test_data


In [19]:
test_ds = ds.SegmentationImageDataset(db_connector=test_con,segmentation_images=test_imgs)
test_loader = DataLoader(test_ds, batch_size,
                             sampler=SequentialSampler(test_ds),
                             num_workers=threads,
                             pin_memory=use_cuda)

In [3]:
ex_img_name = os.listdir(test_folder)[2]
ex_img = cv2.imread(os.path.join(test_folder,ex_img_name))
ex_label_img = cv2.imread(os.path.join(labels_folder,ex_img_name))
cropped = transformer.center_cropping_resize(ex_img,[320,160])
t = torch.from_numpy(cropped.transpose(2,0,1)).float().unsqueeze(0)
res = net(t)[0][1].detach()
plt.subplot(221)
plt.imshow(cropped)
plt.title('Image')
plt.subplot(222)
plt.imshow(ex_label_img)
plt.title('Label')
plt.show()

plt.imshow(res)
plt.title('Result')
plt.show()

NameError: name 'os' is not defined

In [3]:
data_folder = os.environ.get('KITCAR_GAZEBO_SIM_PATH') + '/datasets/segmentation/'
train_folder = data_folder + 'data'
test_folder = data_folder + 'test'
labels_folder = data_folder + 'labels'

In [4]:
test_con.insert_folder(test_folder)

Attempting to insert image 5298c9f0c46a6fe8517f009b57a83b41
Connected as  kitcar to MySQL database  kitcar
INSERT INTO segmentation_test_data (`key`,`small_binary`) VALUES (%s,%s)
Error while executing query:  No result set to fetch from.
Attempting to insert image b216c0170fbf6ff0d89edcbbe26dae0e
Connected as  kitcar to MySQL database  kitcar
INSERT INTO segmentation_test_data (`key`,`small_binary`) VALUES (%s,%s)
Error while executing query:  No result set to fetch from.
Attempting to insert image 61bb9020dc129c7e58a19fa6bde4eb73
Connected as  kitcar to MySQL database  kitcar
INSERT INTO segmentation_test_data (`key`,`small_binary`) VALUES (%s,%s)
Error while executing query:  No result set to fetch from.
Attempting to insert image 554f9cdf198740fbe22d2802d87f7ede
Connected as  kitcar to MySQL database  kitcar
INSERT INTO segmentation_test_data (`key`,`small_binary`) VALUES (%s,%s)
Error while executing query:  No result set to fetch from.


In [5]:
test_folder

'/home/ditschuk/kitcar/kitcar-gazebo-simulation/datasets/segmentation/test'